In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("Lab_3_plotting.ipynb")

## Lab 3: Plotting data

Read in the right hand accelerometer data for subject 1's first clap and plot it for each of the x, y, and z channels. I recommend doing this in the following order:

- Write plotting code for a single dimension. Make sure it's working and the titles/labels etc are correct 
- Once that's working, add a **for** loop to loop over x, y, z
- If doing a **for** loop is confusing, then start with just copying the code over, once, and changing the dimension you're using in the copy. Then go back and put the for loop in

[optional] Do nicer colors than the default ones.

For setting the colors and the data labels, you can either do it by hand, or setup a list of color names and use that.

Slides: https://docs.google.com/presentation/d/1IiGGUNet-4Nj07x2cTXU6IOYXy9TSdAF5OUWCCKIYEM/edit?usp=sharing


In [2]:
# Install a pip package in the current Jupyter kernel
# Note: this is a safety check in case your JN install was not in the same place as your libraries
import sys
!{sys.executable} -m pip install matplotlib

In [3]:
# Libraries that we need to import - numpy and json (for loading the description file)
import numpy as np
import json as json
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Numeric ids to indicate hand motion type from Lab 1
clap_id = 1
high_five_id = 2
snap_id = 3

In [5]:
# TODO: Copy get_descriptor, get_file_info, get_data, get_data_from_files, get_channel_data from lab 2.
#  Put each function in its own cell
def get_descriptor(data_description, name):
    """ Returns the descriptor for the given data channel.
    @param name - The name of the data channel to look for. """

# check if item in data description is desired
    for item in data_description["data_channels"]:
        if item["name"] == name:
            descriptor_dict = item
            break

    return descriptor_dict

    pass

def get_file_info(csv_path):
    """Function that returns a file_info dictionary for a given filepath.
    @param csv_path - path to a CSV file containing a hand motion
    @return A dictionary with key "csv_path", containing csv_path, and "motion_id", containing the type of motion encoded in the file."""

    # get file name from path and find 4th letter
    filename = os.path.basename(csv_path)
    data_letter = filename[3]

    # determine motion_id based on 4th letter
    if data_letter == "C":
        motion_id = clap_id
    elif data_letter == "F":
        motion_id = high_five_id
    elif data_letter == "S":
        motion_id = snap_id

    # create dictionary based on path and motion_id
    data_info = {"csv_path":csv_path, "motion_id":motion_id}

    return data_info

    # data_letter = filename

    # The base file path is of the form S##[C|F|S]##.csv.
    pass

def get_data(file_info):
    """ Function that returns the data from the given CSV file.
    @param file_info - a dictionary with keys "csv_path" and "motion_id"
    @return Return array should contain data in file with an extra column at the end containing the motion_id."""

    # import csv as file_data
    file_data = np.loadtxt(file_info["csv_path"], dtype="float", delimiter=",")

    # get motion id
    motion_id = file_info["motion_id"]

    # create numpy array, add file_data and append motion_id
    file_data_motion = np.zeros((file_data.shape[0], file_data.shape[1]+1))
    file_data_motion[:, 0:-1] = file_data
    file_data_motion[:, -1] = motion_id

    return file_data_motion

    pass

def get_data_from_files(file_list):
    """ Function that returns data from a list of files.
    @param file_list - a list of dictionaries, where each dictionary contains `csv_path` and `motion_id`.
    @return A single return array containing the data from all of the given input files."""

    # create empty list :)
    file_list_list = []

    # based on input list run previous function, concatinate
    for item in file_list:

        data = get_data(item)
        file_list_list.append(data)

    return np.concatenate(file_list_list)


    # Hint: Use np.concatenate to combine multiple numpy arrays.
    pass

def get_channel_data(all_data, index_offset, n_dims):
    """ Get the data for just one channel (e.g., right hand accelerometer)
    @param all_data - numpy array containing data from one or more files
    @param index_offset - the column to begin getting data from
    @param n_dims - number of dimensions for the data channel
    @return Return array should be number of rows in all_data X n_dims"""

    # create array with shape of data and desired dimensions
    channel_data = np.zeros((all_data.shape[0], n_dims))
    # insert desired data into array
    channel_data = all_data[:, index_offset:index_offset + n_dims]

    return channel_data

    pass


In [6]:
# Read in one data file and the data description.
all_data = get_data_from_files([
    get_file_info("Data/S01C01.csv"),
])

try:
    with open("Data/data_description.json", "r") as fp:
        data_description = json.load(fp)
except FileNotFoundError:
    print(f"The file was not found; check that the data directory is in the current one and the file is in that directory")


In [7]:
# import data_description file
try:
    with open("Data/data_description.json", "r") as fp:
        data_description = json.load(fp)
except FileNotFoundError:
    print(f"The file was not found; check that the data directory is in the current one and the file is in that directory")

# get rh_accel descriptor & data
rh_accelerometer_descriptor = get_descriptor(data_description, 'Right hand accelerometer')
rh_accelerometer_data = get_channel_data(all_data, rh_accelerometer_descriptor['index_offset'], rh_accelerometer_descriptor['dimensions'])

# get ts descriptor & data
ts_descriptor = get_descriptor(data_description, 'Timestamp')
ts = get_channel_data(all_data, ts_descriptor['index_offset'], ts_descriptor['dimensions'])

In [8]:
nrows = 1
ncols = 1
_, axs_wf = plt.subplots(nrows, ncols, figsize=(6, 4))

# Useful for labeling the channels with x,y,z instead of 0, 1, 2
channel_labels = ['x', 'y', 'z']

# TODO: Graph the right hand accelerometer data.
# plot data
axs_wf.plot(ts, rh_accelerometer_data[:, 0], label=f'{channel_labels[0]}')
axs_wf.plot(ts, rh_accelerometer_data[:, 1], label=f'{channel_labels[1]}')
axs_wf.plot(ts, rh_accelerometer_data[:, 2], label=f'{channel_labels[2]}')
# set labels
axs_wf.set_title('Right hand accelerometer Data')
axs_wf.set_xlabel('Timestamp (milliseconds)')
axs_wf.set_ylabel('RH Accel Data (gravity units)')
axs_wf.legend(loc='upper right')



In [9]:
grader.check("plot_one")

plot_one results: All test cases passed!

## Part 2 - plot both the right hand gyroscope and right hand accelerometer in the same figure
Gyroscope on the left, Accelerometer on the right. Don't forget labels and titles.

You can do this one of two ways; note that in the homework you'll need to do the 2nd option (with some help), but if you want to try it now, feel free.
- Option 1: Copy and paste the code from above (twice) and change the relevant data/lines of code (axs[], names/titles, start index) in the second copy
- Option 2: Encapsulate the plotting part in a function then call that function twice
  - pass in the axes to plot in, along with t values (or create them within the function)
  - do the x,y,z loop inside of the plot

In [10]:
# Reading the right hand gyroscope data for you
rh_gyroscope_descriptor = get_descriptor(data_description, "Right hand gyroscope")
rh_gyroscope_data = get_channel_data(all_data, index_offset=rh_gyroscope_descriptor["index_offset"], n_dims=rh_gyroscope_descriptor["dimensions"])

In [11]:
def plot_function(axs, ts, data, channel_info, dimension_labels):
    """ Plot all of the dimensions for a given channel
    @param axs - the place to plot in
    @param ts - the time steps to use
    @param data - the numpy array from the csv file
    @param channel_info - a dictionary with the channel info, e.g. "name" and "units"
    @param n_total_dims - total number of columns per one time step
    @returns - nothing"""

    # for each dimension in dataset plot, and set labels
    for r in range(channel_info['dimensions']):
        axs.plot(ts, data[:, r], label=f"{dimension_labels[r]}")
        axs.legend(loc='upper right')
        axs.set_title(f'{channel_info['name']}')
        axs.set_xlabel('Timestamp (milliseconds)')
        axs.set_ylabel(f'{channel_info['name']} ({channel_info['units']})')

    pass

In [12]:
nrows = 1
ncols = 2
fig, axs_two = plt.subplots(nrows, ncols, figsize=(12, 4))

# Call plot function here, once with the gyroscope data, once with the accelerometer data
plot_function(axs = axs_two[1], ts = ts, data = rh_accelerometer_data, channel_info = rh_accelerometer_descriptor, dimension_labels = channel_labels)
plot_function(axs = axs_two[0], ts = ts, data = rh_gyroscope_data, channel_info = rh_gyroscope_descriptor, dimension_labels = channel_labels)

fig.tight_layout()

In [13]:
grader.check("plot_gyroscope_and_accelerometer")

plot_gyroscope_and_accelerometer results: All test cases passed!

### Plotting all data channels

In the previous step, you made plots for the right hand accelerometer and gyroscope. Now, let's make plots for every data channel (except for "Timestamp").

The first column should be the left hand data channels, and the second column should be the right hand data channels. The first row should be the accelerometers, and the second row should be the gyroscopes.

In [14]:
# get description for lh accel, get channel data
lh_accelerometer_descriptor = get_descriptor(data_description, "Left hand accelerometer")
lh_accelerometer_data = get_channel_data(all_data, index_offset=lh_accelerometer_descriptor["index_offset"], n_dims=lh_accelerometer_descriptor["dimensions"])
# get description for lhg, get channel data
lh_gyroscope_descriptor = get_descriptor(data_description, "Left hand gyroscope")
lh_gyroscope_data = get_channel_data(all_data, index_offset=lh_gyroscope_descriptor["index_offset"], n_dims=lh_gyroscope_descriptor["dimensions"])

nrows = 2
ncols = 2
_, axs_all = plt.subplots(nrows, ncols, figsize=(15, 12))

# TODO: Call plot_function for each data channel.
plot_function(axs = axs_all[0, 1], ts = ts, data = rh_accelerometer_data, channel_info = rh_accelerometer_descriptor, dimension_labels = channel_labels)
plot_function(axs = axs_all[1, 1], ts = ts, data = rh_gyroscope_data, channel_info = rh_gyroscope_descriptor, dimension_labels = channel_labels)

plot_function(axs = axs_all[0, 0], ts = ts, data = lh_accelerometer_data, channel_info = lh_accelerometer_descriptor, dimension_labels = channel_labels)
plot_function(axs = axs_all[1, 0], ts = ts, data = lh_gyroscope_data, channel_info = lh_gyroscope_descriptor, dimension_labels = channel_labels)


fig.tight_layout()

In [15]:
grader.check("plot_all")

plot_all results: All test cases passed!

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [16]:

# List of names (creates a set)
worked_with_names = {"N/A"}
# List of URLS I25 (creates a set)
websites = {"N/A"}
# Approximate number of hours, including lab/in-class time
hours = 1

In [17]:
grader.check("hours_collaborators")

hours_collaborators results: All test cases passed!

### To submit

- Make sure your plots are visible and there aren't pages of print outs the TAs have to look through to find them
- Save the file
- Submit just this .ipynb file through gradescope, Lab 3, plotting
- You do NOT need to submit the data files - we will supply those
- Where there are given variable/file names (eg, foo = ...) DON'T change those, or the autograder will fail

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

Most likely failure for this assignment is not naming the data directory and files correctly; capitalization matters for the Gradescope grader. 

Second most likely cause for failure is reading in any files but `data/S01C01.csv` (make sure when you copied your code over you didn't accidentally include code that reads in other files)

Several of these problems are manually graded by looking at the plots. If we can't see the plots, we can't grade them.